In [28]:
import pandas as pd

In [29]:
df = pd.read_parquet('data/yellow_tripdata_2022-01.parquet').head(1000)  # sampling data
df_taxi_zone = pd.read_csv('data/taxi_zones.csv')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [30]:
def get_coordinates(id, column='x'):
    try:
        return df_taxi_zone[df_taxi_zone['location_id'] == id][[f'coordinate_{column}']].values[0][0]
    except IndexError:
        return 0

In [31]:
get_coordinates(3), get_coordinates(2024)  #Testing Function

(-73.84742223236718, 0)

In [32]:
df_taxi_zone

,location_id,zone,coordinate_x,coordinate_y
0,1,Newark Airport,-74.174000,40.691831
1,2,Jamaica Bay,-73.831299,40.616745
2,3,Allerton/Pelham Gardens,-73.847422,40.864474
3,4,Alphabet City,-73.976968,40.723752
4,5,Arden Heights,-74.188484,40.552659
...,...,...,...,...
258,256,Williamsburg (South Side),-73.959905,40.710880
259,259,Woodlawn/Wakefield,-73.852215,40.897932
260,260,Woodside,-73.906306,40.744235
261,261,World Trade Center,-74.013023,40.709139


In [33]:
df["PULocation_x"]= df.apply(lambda row: get_coordinates(row["PULocationID"]), axis=1)
df["PULocation_y"]= df.apply(lambda row: get_coordinates(row["PULocationID"], 'y'), axis=1)
df["DOLocation_x"]= df.apply(lambda row: get_coordinates(row["DOLocationID"]), axis=1)
df["DOLocation_y"]= df.apply(lambda row: get_coordinates(row["PULocationID"],'y'), axis=1)

df.drop(['PULocationID', 'DOLocationID'], axis=1, inplace=True)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               1000 non-null   int64         
 1   tpep_pickup_datetime   1000 non-null   datetime64[ns]
 2   tpep_dropoff_datetime  1000 non-null   datetime64[ns]
 3   passenger_count        1000 non-null   float64       
 4   trip_distance          1000 non-null   float64       
 5   RatecodeID             1000 non-null   float64       
 6   store_and_fwd_flag     1000 non-null   object        
 7   payment_type           1000 non-null   int64         
 8   fare_amount            1000 non-null   float64       
 9   extra                  1000 non-null   float64       
 10  mta_tax                1000 non-null   float64       
 11  tip_amount             1000 non-null   float64       
 12  tolls_amount           1000 non-null   float64       
 13  impr

In [35]:
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])

In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               1000 non-null   int64         
 1   tpep_pickup_datetime   1000 non-null   datetime64[ns]
 2   tpep_dropoff_datetime  1000 non-null   datetime64[ns]
 3   passenger_count        1000 non-null   float64       
 4   trip_distance          1000 non-null   float64       
 5   RatecodeID             1000 non-null   float64       
 6   store_and_fwd_flag     1000 non-null   object        
 7   payment_type           1000 non-null   int64         
 8   fare_amount            1000 non-null   float64       
 9   extra                  1000 non-null   float64       
 10  mta_tax                1000 non-null   float64       
 11  tip_amount             1000 non-null   float64       
 12  tolls_amount           1000 non-null   float64       
 13  impr

In [37]:
df = df.drop_duplicates().reset_index(drop=True)
df['trip_id'] = df.index

In [38]:
datetime_dim = df[['tpep_pickup_datetime', 'tpep_dropoff_datetime']].reset_index(drop=True)
datetime_dim['pick_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['pick_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['pick_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['pick_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['pick_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday


datetime_dim['drop_hour'] = datetime_dim['tpep_pickup_datetime'].dt.hour
datetime_dim['drop_day'] = datetime_dim['tpep_pickup_datetime'].dt.day
datetime_dim['drop_month'] = datetime_dim['tpep_pickup_datetime'].dt.month
datetime_dim['drop_year'] = datetime_dim['tpep_pickup_datetime'].dt.year
datetime_dim['drop_weekday'] = datetime_dim['tpep_pickup_datetime'].dt.weekday

datetime_dim

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,2022-01-01 00:35:40,2022-01-01 00:53:29,0,1,1,2022,5,0,1,1,2022,5
1,2022-01-01 00:33:43,2022-01-01 00:42:07,0,1,1,2022,5,0,1,1,2022,5
2,2022-01-01 00:53:21,2022-01-01 01:02:19,0,1,1,2022,5,0,1,1,2022,5
3,2022-01-01 00:25:21,2022-01-01 00:35:23,0,1,1,2022,5,0,1,1,2022,5
4,2022-01-01 00:36:48,2022-01-01 01:14:20,0,1,1,2022,5,0,1,1,2022,5
...,...,...,...,...,...,...,...,...,...,...,...,...
995,2022-01-01 00:15:15,2022-01-01 00:19:08,0,1,1,2022,5,0,1,1,2022,5
996,2022-01-01 00:32:25,2022-01-01 00:44:22,0,1,1,2022,5,0,1,1,2022,5
997,2022-01-01 00:53:58,2022-01-01 00:57:04,0,1,1,2022,5,0,1,1,2022,5
998,2022-01-01 00:21:20,2022-01-01 00:32:00,0,1,1,2022,5,0,1,1,2022,5


In [39]:
datetime_dim['datetime_id'] = datetime_dim.index

In [40]:
datetime_dim = datetime_dim[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
       'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

In [41]:
datetime_dim

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2022-01-01 00:35:40,0,1,1,2022,5,2022-01-01 00:53:29,0,1,1,2022,5
1,1,2022-01-01 00:33:43,0,1,1,2022,5,2022-01-01 00:42:07,0,1,1,2022,5
2,2,2022-01-01 00:53:21,0,1,1,2022,5,2022-01-01 01:02:19,0,1,1,2022,5
3,3,2022-01-01 00:25:21,0,1,1,2022,5,2022-01-01 00:35:23,0,1,1,2022,5
4,4,2022-01-01 00:36:48,0,1,1,2022,5,2022-01-01 01:14:20,0,1,1,2022,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2022-01-01 00:15:15,0,1,1,2022,5,2022-01-01 00:19:08,0,1,1,2022,5
996,996,2022-01-01 00:32:25,0,1,1,2022,5,2022-01-01 00:44:22,0,1,1,2022,5
997,997,2022-01-01 00:53:58,0,1,1,2022,5,2022-01-01 00:57:04,0,1,1,2022,5
998,998,2022-01-01 00:21:20,0,1,1,2022,5,2022-01-01 00:32:00,0,1,1,2022,5


In [43]:
passenger_count_dim = df[['passenger_count']].reset_index(drop=True)
passenger_count_dim['passenger_count_id'] = passenger_count_dim.index
passenger_count_dim['passenger_count'] = passenger_count_dim['passenger_count'].astype('int')
passenger_count_dim[['passenger_count_id', 'passenger_count']]

,passenger_count_id,passenger_count
0,0,2
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
995,995,2
996,996,3
997,997,2
998,998,1


In [45]:
trip_distance_dim = df[['trip_distance']].reset_index(drop=True)
trip_distance_dim['trip_distance_id'] = trip_distance_dim.index
trip_distance_dim[['trip_distance_id', 'trip_distance']]

,trip_distance_id,trip_distance
0,0,3.80
1,1,2.10
2,2,0.97
3,3,1.09
4,4,4.30
...,...,...
995,995,0.98
996,996,2.13
997,997,0.35
998,998,1.85


In [46]:
PU_location_dim = df[['PULocation_x', 'PULocation_y']].reset_index(drop=True)
PU_location_dim['PU_location_id'] = PU_location_dim.index
PU_location_dim[['PU_location_id', 'PULocation_x', 'PULocation_y']]

,PU_location_id,PULocation_x,PULocation_y
0,0,-73.981532,40.773633
1,1,-73.957012,40.780436
2,2,-73.961764,40.809457
3,3,-73.997380,40.728340
4,4,-73.999917,40.748428
...,...,...,...
995,995,-73.985937,40.727620
996,996,-73.985937,40.727620
997,997,-73.984052,40.736824
998,998,-73.985156,40.748575


In [47]:
DO_location_dim = df[['DOLocation_x', 'DOLocation_y']].reset_index(drop=True)
DO_location_dim['DO_location_id'] = DO_location_dim.index
DO_location_dim[['DO_location_id', 'DOLocation_x', 'DOLocation_y']]

,DO_location_id,DOLocation_x,DOLocation_y
0,0,-73.957012,40.773633
1,1,-73.940772,40.780436
2,2,-73.961764,40.809457
3,3,-73.999917,40.728340
4,4,-73.977569,40.748428
...,...,...,...
995,995,-73.978492,40.727620
996,996,-73.970443,40.727620
997,997,-73.984052,40.736824
998,998,-74.002875,40.748575


In [51]:
payment_type_dim = df[['payment_type']].reset_index(drop=True)
payment_type_dim['payment_type_id'] = payment_type_dim.index
payment_name_dict = {1: 'Credit card', 2: 'Cash', 3: 'No charge', 4: 'Dispute', 5: 'Unknown', 6: 'Voided trip'}
payment_type_dim['payment_type_name'] = payment_type_dim['payment_type'].map(payment_name_dict)
payment_type_dim[['payment_type_id', 'payment_type', 'payment_type_name']]

,payment_type_id,payment_type,payment_type_name
0,0,1,Credit card
1,1,1,Credit card
2,2,1,Credit card
3,3,2,Cash
4,4,1,Credit card
...,...,...,...
995,995,1,Credit card
996,996,2,Cash
997,997,2,Cash
998,998,1,Credit card


In [52]:
rate_code_dim = df[['RatecodeID']].reset_index(drop=True)
rate_code_dim['rate_code_id'] = rate_code_dim.index
rate_code_name_dict = {1: 'Standard rate', 2: 'JFK', 3: 'Newark', 4: 'Nassau or Westchester', 5: 'Negotiated fare', 6: 'Group ride'}
rate_code_dim['rate_code_name'] = rate_code_dim['RatecodeID'].map(rate_code_name_dict)
rate_code_dim[['rate_code_id', 'RatecodeID', 'rate_code_name']]

,rate_code_id,RatecodeID,rate_code_name
0,0,1.0,Standard rate
1,1,1.0,Standard rate
2,2,1.0,Standard rate
3,3,1.0,Standard rate
4,4,1.0,Standard rate
...,...,...,...
995,995,1.0,Standard rate
996,996,1.0,Standard rate
997,997,1.0,Standard rate
998,998,1.0,Standard rate


In [54]:
fact_table = df.merge(passenger_count_dim, left_on='trip_id', right_on='passenger_count_id').merge(trip_distance_dim, left_on='trip_id', right_on='trip_distance_id').merge(rate_code_dim, left_on='trip_id', right_on='rate_code_id').merge(PU_location_dim, left_on='trip_id', right_on='PU_location_id').merge(DO_location_dim, left_on='trip_id', right_on='DO_location_id').merge(datetime_dim, left_on='trip_id', right_on='datetime_id').merge(payment_type_dim, left_on='trip_id', right_on='payment_type_id')[['trip_id','VendorID', 'datetime_id', 'passenger_count_id',
               'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag', 'PU_location_id', 'DO_location_id',
               'payment_type_id', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

In [55]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,PU_location_id,DO_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,14.5,3.0,0.5,3.65,0.0,0.3,21.95
1,1,1,1,1,1,1,N,1,1,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30
2,2,2,2,2,2,2,N,2,2,2,7.5,0.5,0.5,1.76,0.0,0.3,10.56
3,3,2,3,3,3,3,N,3,3,3,8.0,0.5,0.5,0.00,0.0,0.3,11.80
4,4,2,4,4,4,4,N,4,4,4,23.5,0.5,0.5,3.00,0.0,0.3,30.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,2,995,995,995,995,N,995,995,995,5.0,0.5,0.5,1.76,0.0,0.3,10.56
996,996,2,996,996,996,996,N,996,996,996,10.0,0.5,0.5,0.00,0.0,0.3,13.80
997,997,2,997,997,997,997,N,997,997,997,4.0,0.5,0.5,0.00,0.0,0.3,7.80
998,998,2,998,998,998,998,N,998,998,998,9.0,0.5,0.5,1.00,0.0,0.3,13.80
